# Projet SDP : Explications de classements par Trade-offs

**Membres du groupe** : Adham Noureldin, Aymane Chalh, Daphné Maschas

**Date** : 19 Janvier 2026

In [ ]:
import gurobipy as gp
from gurobipy import GRB

## 1. Introduction

Ce projet vise à fournir des explications intelligibles pour le classement des candidats au concours de l'internat de médecine. Au lieu d'utiliser la somme pondérée brute, nous justifions qu'un candidat $x$ est meilleur qu'un candidat $y$ en montrant comment ses points forts compensent ses points faibles via des modèles d'optimisation (MIP).

## 2. Préparation des données

Dans cette section, nous définissons les poids des matières et les notes des candidats de référence.

In [20]:
matieres = ["Anatomie", "Biologie", "Chirurgie", "Diagnostic", "Epidemiologie", "Forensic Pathology", "Génétique"]
poids = [8, 7, 7, 6, 6, 5, 6]

# Notes des candidats 
notes_x = [85, 81, 71, 69, 75, 81, 88] # Xavier
notes_y = [81, 81, 75, 63, 67, 88, 95] # Yvonne
notes_z = [74, 89, 74, 81, 68, 84, 79]
notes_t = [74, 71, 84, 91, 77, 76, 73]
notes_w = [79, 69, 78, 76, 67, 84, 79]
notes_w_prime = [57, 76, 81, 76, 82, 86, 77]
notes_u = [72, 66, 75, 85, 88, 66, 93]
notes_v = [71, 73, 63, 92, 76, 79, 93]

# Autres candidats
notes_a1 = [89, 74, 81, 68, 84, 79, 77]
notes_a2 = [71, 84, 91, 79, 78, 73.5, 77] # 73.5 pour Forensic

Fonctions utilitaires:

In [24]:
def preparer_donnees(notes_a, notes_b):
    """Calcule les contributions et sépare les Pros et les Cons."""
    contribs = [poids[i] * (notes_a[i] - notes_b[i]) for i in range(len(poids))]
    P = [i for i, c in enumerate(contribs) if c > 0]
    C = [i for i, c in enumerate(contribs) if c < 0]
    return contribs, P, C

def afficher_resultats(titre, status, explication):
    """Affiche proprement le résultat d'un modèle d'optimisation."""
    print(f"\n=== {titre} ===")
    if status == "OPTIMAL" and explication:
        print("SUCCÈS : Une explication a été trouvée :")
        for groupe in explication:
            print(f"  • {groupe}")
    else:
        print("ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.")
    print("-" * 50)

## 3. Question 1 : Explication de type (1-1)

**Démarche** : Nous cherchons à coupler chaque critère défavorable à un unique critère favorable qui le compense individuellement.

**Contrainte** : Chaque "Pro" est utilisé au plus une fois, chaque "Con" est couvert une fois.

**Condition** : $c_{pro} + c_{con} > 0$.

In [26]:
def explication_1_1(notes_a, notes_b):
    contribs, P, C = preparer_donnees(notes_a, notes_b)
    
    m = gp.Model("Q1_1_1")
    m.Params.OutputFlag = 0
    
    # x[i,j] = 1 si le pro i compense le con j
    x = m.addVars(P, C, vtype=GRB.BINARY, name="x")
    
    # Contraintes : Couplage parfait
    for j in C:
        m.addConstr(gp.quicksum(x[i,j] for i in P) == 1)
    for i in P:
        m.addConstr(gp.quicksum(x[i,j] for j in C) <= 1)
        
    # Validité : pro + con > 0
    for i in P:
        for j in C:
            if contribs[i] + contribs[j] <= 0:
                m.addConstr(x[i,j] == 0)

    m.optimize()
    
    if m.Status == GRB.OPTIMAL:
        explication = [f"{matieres[i]} (+{contribs[i]}) compense {matieres[j]} ({contribs[j]})" 
                       for i in P for j in C if x[i,j].X > 0.5]
        return "OPTIMAL", explication
    return "INFEASIBLE", None

In [29]:
status, explication = explication_1_1(notes_x, notes_y)
afficher_resultats("QUESTION 1 (1-1) : Xavier vs Yvonne", status, explication)


=== QUESTION 1 (1-1) : Xavier vs Yvonne ===
SUCCÈS : Une explication a été trouvée :
  • Anatomie (+32) compense Chirurgie (-28)
  • Diagnostic (+36) compense Forensic Pathology (-35)
  • Epidemiologie (+48) compense Génétique (-42)
--------------------------------------------------


## 4. Question 2: Explication (1-m)

**Démarche** : Nous généralisons le modèle.

**Type (1-m)** : Un avantage puissant peut compenser plusieurs faiblesses. C'est utile quand un candidat excelle dans une matière à gros coefficient.

In [30]:
def explication_1_m(notes_a, notes_b, poids, matieres, epsilon=1e-4):
    """
    Résout le problème d'explication de type (1-m).
    Retourne le statut et l'explication si trouvée.
    """
    # Calcul des contributions
    contribs, P, C = preparer_donnees(notes_a, notes_b)
    
    # Modèle Gurobi
    m = gp.Model("Explication_1_m")
    m.setParam('OutputFlag', 0) # Mode silencieux

    # Variables
    # x[i,j] = 1 si Pro i couvre Con j
    x = {}
    for i in P:
        for j in C:
            x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"link_{i}_{j}")
    
    # y[i] = 1 si Pro i est utilisé
    y = {}
    for i in P:
        y[i] = m.addVar(vtype=GRB.BINARY, name=f"use_{i}")

    # Contrainte C1: Chaque Con doit être couvert exactement une fois
    for j in C:
        m.addConstr(gp.quicksum(x[i, j] for i in P) == 1, name=f"cover_{j}")

    # Contrainte C2: Lien x_ij <= y_i
    for i in P:
        for j in C:
            m.addConstr(x[i, j] <= y[i], name=f"link_logic_{i}_{j}")

    # Contrainte C3: Somme pondérée positive pour chaque groupe (1-m)
    # delta_i * y_i + sum(contribs_j * x_ij) >= epsilon * y_i
    for i in P:
        sum_cons = gp.quicksum(contribs[j] * x[i, j] for j in C)
        m.addConstr(contribs[i] * y[i] + sum_cons >= epsilon * y[i], name=f"validity_{i}")

    # Résolution
    m.optimize()

    # Analyse
    if m.Status == GRB.OPTIMAL:
        explanation = {}
        for i in P:
            if y[i].X > 0.5:
                covered_cons = [matieres[j] for j in C if x[i, j].X > 0.5]
                explanation[matieres[i]] = covered_cons
        return "OPTIMAL", explanation
    elif m.Status == GRB.INFEASIBLE:
        return "INFEASIBLE", None
    else:
        return f"Status {m.Status}", None

In [33]:
status, result = explication_1_m(notes_w, notes_w_prime, poids, matieres)
afficher_resultats("QUESTION 2 (1-m) : w vs w'", status, explication)


=== QUESTION 2 (1-m) : w vs w' ===
SUCCÈS : Une explication a été trouvée :
  • Anatomie (+32) compense Chirurgie (-28)
  • Diagnostic (+36) compense Forensic Pathology (-35)
  • Epidemiologie (+48) compense Génétique (-42)
--------------------------------------------------


In [34]:
status, result = explication_1_m(notes_u, notes_v, poids, matieres)
afficher_resultats("QUESTION 2 (1-m) : u vs v", status, explication)


=== QUESTION 2 (1-m) : u vs v ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------


## 5. Question 3: Explication (m-1)

**Démarche** : Nous généralisons le modèle.

**Type (m-1)** : Plusieurs petits avantages s'unissent pour compenser une faiblesse majeure.

In [38]:
def explication_m_1(candidate_sup, candidate_inf, notes_a, notes_b, poids, matieres):
    """
    Résout le problème d'explication de type (m-1).
    Plusieurs 'Pros' peuvent compenser un seul 'Con'.
    """
    # Calcul des contributions
    contribs, P, C = preparer_donnees(notes_a, notes_b)

    # Modèle Gurobi
    m = gp.Model(f"Explication_m_1_{candidate_sup}_{candidate_inf}")
    m.setParam('OutputFlag', 0)

    # Variables x[i, j] : Pro i aide à couvrir Con j
    x = {}
    for i in P:
        for j in C:
            x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"x_{matieres[i]}_{matieres[j]}")

    # C1: Chaque Pro utilisé au max 1 fois
    for i in P:
        m.addConstr(gp.quicksum(x[i, j] for j in C) <= 1, name=f"disjoint_{matieres[i]}")

    # C2: Chaque Con doit être compensé (Somme Pros + Con >= 0)
    # On utilise une tolérance très faible ou nulle. Ici >= 0 accepte l'égalité parfaite.
    for j in C:
        m.addConstr(gp.quicksum(contribs[i] * x[i, j] for i in P) + contribs[j] >= 0, 
                    name=f"cover_{matieres[j]}")

    # Résolution
    m.optimize()

    # Analyse
    if m.Status == GRB.OPTIMAL:
        res = []
        for j in C:
            # Récupérer les Pros assignés à ce Con j
            avantages = [matieres[i] for i in P if x[i,j].X > 0.5]
            res.append(f"L'union de {avantages} compense {matieres[j]} ({contribs[j]})")
        return "OPTIMAL", res
    return "INFEASIBLE", None

In [43]:
# Test 1 : y > z
status, result = explication_m_1("y", "z", notes_y, notes_z, poids, matieres)
afficher_resultats("QUESTION 3 (m-1) : Candidat u vs v", status, result)


=== QUESTION 3 (m-1) : Candidat u vs v ===
SUCCÈS : Une explication a été trouvée :
  • L'union de ['Anatomie'] compense Biologie (-56)
  • L'union de ['Forensic Pathology', 'Génétique'] compense Diagnostic (-108)
  • L'union de ['Chirurgie'] compense Epidemiologie (-6)
--------------------------------------------------


In [42]:
status, result = explication_m_1("z", "t", notes_z, notes_t, poids, matieres)
afficher_resultats("QUESTION 3 (m-1) : Candidat u vs v", status, result)


=== QUESTION 3 (m-1) : Candidat u vs v ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------


## 6. Question 4: Modèle Hybride

**Démarche** : Certains cas, comme $u > v$, ne possèdent pas d'explication simple. Nous implémentons un modèle hybride qui autorise une partition arbitraire des critères en groupes (1-m) ou (m-1). Nous utilisons une formulation avec des variables binaires et des contraintes de type "Big-M" pour activer les trade-offs.

In [ ]:
def explication_hybride(notes_a, notes_b):
    """
    Résout le problème d'explication mixte : autorise les structures (1-m) OU (m-1).
    """
    # Calcul des contributions
    contribs, P, C = preparer_donnees(notes_a, notes_b)

    # Modèle
    m = gp.Model("Mixed_Explanation")
    m.setParam('OutputFlag', 0)
    
    # Pour le type (1-m) : Pro i est le "Hub" qui couvre les Cons j (Leaves)
    # x_1m[i,j] = 1 si i couvre j dans un schéma (1-m)
    x_1m = {} 
    
    # Pour le type (m-1) : Con j est le "Hub" couvert par les Pros i (Leaves)
    # x_m1[i,j] = 1 si i aide à couvrir j dans un schéma (m-1)
    x_m1 = {}

    for i in P:
        for j in C:
            x_1m[i, j] = m.addVar(vtype=GRB.BINARY, name=f"1m_{matieres[i]}_{matieres[j]}")
            x_m1[i, j] = m.addVar(vtype=GRB.BINARY, name=f"m1_{matieres[i]}_{matieres[j]}")

    # Indicateurs de rôle "Hub"
    # h_P[i] = 1 si le Pro i est utilisé comme centre d'un (1-m)
    h_P = {i: m.addVar(vtype=GRB.BINARY) for i in P}
    
    # h_C[j] = 1 si le Con j est utilisé comme centre d'un (m-1)
    h_C = {j: m.addVar(vtype=GRB.BINARY) for j in C}

    # Contraintes

    # 1. Tout Con doit être couvert exactement une fois
    for j in C:
        m.addConstr(gp.quicksum(x_1m[i, j] for i in P) + h_C[j] == 1, name=f"Cover_{j}")

    # 2. Tout Pro utilisé au max une fois
    for i in P:
        m.addConstr(h_P[i] + gp.quicksum(x_m1[i, j] for j in C) <= 1, name=f"Use_{i}")

    # 3. Cohérence logique des liens
    for i in P:
        for j in C:
            m.addConstr(x_1m[i, j] <= h_P[i])
            
    # Si lien (m-1) i->j existe, alors j doit être déclaré Hub
    for i in P:
        for j in C:
            m.addConstr(x_m1[i, j] <= h_C[j])

    # 4. Validité Financière (Somme pondérée > 0)
    # Pour chaque Pro Hub (Structure 1-m): delta_i + sum(delta_j * x_1m_ij) >= 0
    for i in P:
        contrib_cons = gp.quicksum(contribs[j] * x_1m[i, j] for j in C)
        # On multiplie par h_P[i] pour désactiver la contrainte si pas hub (big-M ou juste logique 0>=0)
        # Ici : delta_i * h_P[i] + contrib_cons >= 0 (car x_ij est 0 si h_P est 0)
        m.addConstr(contribs[i] * h_P[i] + contrib_cons >= 0, name=f"Valid_1m_{i}")

    # Pour chaque Con Hub (Structure m-1)
    # sum(delta_i * x_m1_ij) + delta_j >= 0
    for j in C:
        contrib_pros = gp.quicksum(contribs[i] * x_m1[i, j] for i in P)
        m.addConstr(contrib_pros + contribs[j] * h_C[j] >= 0, name=f"Valid_m1_{j}")

    # Résolution
    m.optimize()

    # Analyse
    if m.Status == GRB.OPTIMAL:
        res = []
        # Structures (1-m)
        for i in P:
            if h_P[i].X > 0.5:
                cons_covered = [matieres[j] for j in C if x_1m[i, j].X > 0.5]
                if cons_covered:
                    res.append(f"Groupe (1-m) : L'avantage {matieres[i]} (+{contribs[i]}) compense {cons_covered}")
        
        # Structures (m-1)
        for j in C:
            if h_C[j].X > 0.5:
                pros_used = [matieres[i] for i in P if x_m1[i, j].X > 0.5]
                val_pros = sum(contribs[i] for i in P if x_m1[i, j].X > 0.5)
                res.append(f"Groupe (m-1) : Les avantages {pros_used} (+{val_pros}) compensent {matieres[j]} ({contribs[j]})")
        
        return "OPTIMAL", res
    
    return "INFEASIBLE", None

In [53]:
status, result = explication_hybride(notes_z, notes_t)
afficher_resultats("QUESTION 4 (hybride) : Candidat z vs t", status, result)


=== QUESTION 4 (hybride) : Candidat z vs t ===
SUCCÈS : Une explication a été trouvée :
  • Groupe (1-m) : L'avantage Biologie (+126) compense ['Diagnostic', 'Epidemiologie']
  • Groupe (m-1) : Les avantages ['Forensic Pathology', 'Génétique'] (+76) compensent Chirurgie (-70)
--------------------------------------------------


In [54]:
status, result = explication_hybride(notes_a1, notes_a2)
afficher_resultats("QUESTION 4 (hybride) : Candidat a1 vs a2", status, result)


=== QUESTION 4 (hybride) : Candidat a1 vs a2 ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------


## 6. Application aux données réelles (Cancer du Sein)

TODO